<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab)V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 1907, done.
remote: Counting objects: 100% (725/725), done.
remote: Compressing objects: 100% (267/267), done.
remote: Total 1907 (delta 472), reused 687 (delta 449), pack-reused 1182
Receiving objects: 100% (1907/1907), 44.65 MiB | 33.62 MiB/s, done.
Resolving deltas: 100% (1250/1250), done.


# import

In [2]:
from c620.FV2 import FV2
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# functions

In [3]:
def check_two_data(d1,d2):
  if np.allclose(d1.values,d2.values):
    print('一樣')
  else:
    print('不一樣')

# demo data

In [4]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
096-002,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,92.0,0.4524,44.244236,44.786339,90.0,860.0,0.010000,10.000091,36.000000
045-011,200.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,57.0,0.2128,35.413631,57.973057,80.0,980.0,0.010000,9.999991,36.000000
090-002,188.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,92.0,0.4524,44.244236,44.786339,90.0,860.0,0.010000,9.999921,36.000000
009-023,175.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,57.0,0.4524,44.244236,44.786339,70.0,950.0,0.010000,10.000003,36.000000
049-002,163.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,32.0,0.0911,20.854397,74.689774,90.0,980.0,4.250001,5.000001,36.000031


In [5]:
idx1 = icg_input.index[0]
idx2 = icg_input.index[1]

# 1.確保給定不同的輸入 會有不同的輸出

In [6]:

f = FV2(config)

# 輸入1
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f.inference(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

# 輸入2
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2 = f.inference(icg_input.loc[[idx2]],c620_feed.loc[[idx2]],t651_feed.loc[[idx2]])

In [7]:
check_two_data(c620_wt1,c620_wt2)
check_two_data(c620_op1,c620_op2)
check_two_data(c660_wt1,c660_wt2)
check_two_data(c660_op1,c660_op2)
check_two_data(c670_wt1,c670_wt2)
check_two_data(c670_op1,c670_op2)

不一樣
不一樣
不一樣
不一樣
不一樣
不一樣


# 2.確保變更模式 推薦 和 試算 模式 有不同的輸出

In [8]:
# 試算模式
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f.inference(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])

# 推薦模式
f.Recommended_mode = True #(True or False)
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2,bz_error,nainbz_error,tol_error = f.recommend(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]],return_error=True)

[I 2021-04-20 05:17:29,535] A new study created in memory with name: no-name-d7ce76e5-fd27-4892-ae90-ff2bd0fbabe8
[I 2021-04-20 05:17:29,578] Trial 0 finished with value: 6.157079261938598 and parameters: {'Density_Feed Properties': 0.8639889464537858, 'Density_Vent Gas Production Rate and Composition': 0.4319470082439471, 'Density_Distillate Production Rate and Composition': 0.8082129573168364, 'Density_Sidedraw Production Rate and Composition': 0.8787209905015874, 'Density_Bottoms Production Rate and Composition': 0.872105509603302, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 114.69205418614874, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 10.662324842682938, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 15.796485420083329, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 175.2662062130186, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 192.

bz_error: 2.391575858998806


[I 2021-04-20 05:17:50,130] Trial 3 finished with value: 70384.65462821933 and parameters: {'Density_Feed Properties': 0.8786569318920284, 'Density_Vent Gas Production Rate and Composition': 0.5692418127251021, 'Density_Distillate (Benzene Drag) Production Rate and Composition': 0.8443028132254151, 'Density_Sidedraw (Benzene )Production Rate and Composition': 0.8837278154555228, 'Density_Bottoms Production Rate and Composition': 0.8700532635135502, 'Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr': 119.31258040144291, 'Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 7.666456166031443, 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 16.44610653936107, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 87.35853548301822, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 88.58191656634251}. Best is trial 3 with value: 70384.65462821933.
[I 2021-04-20 05:17:50,179] Trial

nainbz_error: 255.04155365006557
tol_error: 22.926714662553913


In [9]:
check_two_data(c620_wt1,c620_wt2)
check_two_data(c620_op1,c620_op2)
check_two_data(c660_wt1,c660_wt2)
check_two_data(c660_op1,c660_op2)
check_two_data(c670_wt1,c670_wt2)
check_two_data(c670_op1,c670_op2)

不一樣
不一樣
不一樣
不一樣
不一樣
不一樣


In [10]:
bz_error,nainbz_error,tol_error

(2.391575858998806, 255.04155365006557, 22.926714662553913)

# 確認icg_input 對應到後續系統的輸出值一致 主要有三項要一致

In [11]:
# 試算模式
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
print(icg_input.loc[[idx1],['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']])
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f.inference(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
print(c660_wt1.filter(regex='Sidedraw').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]].sum(axis=1)*10000)
print(c660_wt1['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']*10000)

         Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
096-002                                               90.0               
         Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
096-002                                              860.0     
         Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
096-002                                          10.000091                            
         Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
096-002                                          87.526665                                   
096-002    666.222051
dtype: float64
096-002    11.731441
Name: Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%, dtype: float64


In [12]:
# 推薦模式
c620_wt1,c620_op1,c660_wt1,c660_op1,c670_wt1,c670_op1 = f.recommend(icg_input.loc[[idx1]],c620_feed.loc[[idx1]],t651_feed.loc[[idx1]])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']])
print(icg_input.loc[[idx1],['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']])
print(icg_input.loc[[idx1],['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']])
print(c620_wt1.filter(regex='Sidedraw Production Rate and Composition_Benzene_wt%'))
print(c660_wt1.filter(regex='Sidedraw').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29]].sum(axis=1)*10000)
print(c660_wt1['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']*10000)

[I 2021-04-20 05:18:11,857] A new study created in memory with name: no-name-1563e6e2-2600-4b44-8540-88e3a6864ce7
[I 2021-04-20 05:18:11,901] Trial 0 finished with value: 6.147701380014591 and parameters: {'Density_Feed Properties': 0.8623886703331579, 'Density_Vent Gas Production Rate and Composition': 0.4400397918391509, 'Density_Distillate Production Rate and Composition': 0.801015909771069, 'Density_Sidedraw Production Rate and Composition': 0.8804029133111868, 'Density_Bottoms Production Rate and Composition': 0.8720496189748534, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 142.80854114022117, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 14.369288895232932, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 15.257225236848917, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 175.32826726994566, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193

bz_error: 2.3903191673355906


[I 2021-04-20 05:18:32,434] Trial 3 finished with value: 89054.2833905753 and parameters: {'Density_Feed Properties': 0.8769884048581623, 'Density_Vent Gas Production Rate and Composition': 0.572319349804241, 'Density_Distillate (Benzene Drag) Production Rate and Composition': 0.8585798220597248, 'Density_Sidedraw (Benzene )Production Rate and Composition': 0.8837446717667284, 'Density_Bottoms Production Rate and Composition': 0.8704586369346486, 'Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr': 181.63250147628082, 'Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 8.941334765076089, 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 12.214166203298513, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 86.60518039755202, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 90.23861755953307}. Best is trial 3 with value: 89054.2833905753.
[I 2021-04-20 05:18:32,487] Trial 4

nainbz_error: 259.46667344242576
tol_error: 20.52317793944257
         Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
096-002                                               90.0               
         Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
096-002                                              860.0     
         Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
096-002                                          10.000091                            
         Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
096-002                                          87.609681                                   
096-002    600.533327
dtype: float64
096-002    30.523269
Name: Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%, dtype: float64


# 操作條件預測太準的問題

In [17]:
idx = demo['icg_input'].sample(1).index[0]
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.recommend(demo['icg_input'].loc[[idx]],demo['c620_feed'].loc[[idx]],demo['t651_feed'].loc[[idx]])

[I 2021-04-20 05:20:30,222] A new study created in memory with name: no-name-c5f548f6-0cf8-49ef-87a3-51932e4d7707
[I 2021-04-20 05:20:30,267] Trial 0 finished with value: 56.94921969153353 and parameters: {'Density_Feed Properties': 0.8634847797514368, 'Density_Vent Gas Production Rate and Composition': 0.45039096167353443, 'Density_Distillate Production Rate and Composition': 0.7806802383244569, 'Density_Sidedraw Production Rate and Composition': 0.8815184027828311, 'Density_Bottoms Production Rate and Composition': 0.8721083592997031, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 104.64703736198713, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 14.023296451437435, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 16.653272524670736, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 176.28526125068467, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 1

bz_error: 4.303751062036298


[I 2021-04-20 05:20:50,882] Trial 3 finished with value: 14602.137196806389 and parameters: {'Density_Feed Properties': 0.8780045334864147, 'Density_Vent Gas Production Rate and Composition': 0.5601823937362224, 'Density_Distillate (Benzene Drag) Production Rate and Composition': 0.8481769131037475, 'Density_Sidedraw (Benzene )Production Rate and Composition': 0.8836859860547454, 'Density_Bottoms Production Rate and Composition': 0.87067430964123, 'Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr': 191.5134669280082, 'Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 10.64152675377977, 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 12.447403080226001, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 86.99443435509889, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 92.31130988616806}. Best is trial 1 with value: 13956.345011616619.
[I 2021-04-20 05:20:50,935] Trial

nainbz_error: 60.89458365498376
tol_error: 17.99749687537542


In [18]:
pd.concat([c620_op.iloc[:,-2:],demo['c620_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
095-017,178.018782,194.739639
095-017,177.287994,192.725250


In [19]:
pd.concat([c660_op.iloc[:,-2:],demo['c660_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
095-017,87.179875,92.516221
095-017,86.902069,90.730774


In [20]:
pd.concat([c670_op.iloc[:,-2:],demo['c670_op'].loc[[idx]].iloc[:,-2:]],axis=0)

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
095-017,181.413712,228.39119
095-017,181.610596,228.68338
